In [11]:
import torch
from torch import nn

# Import torchvision 
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

# Import matplotlib for visualization
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

In [17]:
from PIL import Image
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os
data_dir = 'data/'  # Veri klasörünün yolu
batch_size = 128  # Mini-batch boyutu

# Veri dönüşümleri ve etiketleme işlemi
transform = transforms.Compose([
    transforms.Resize((224, 224)),   
    transforms.ToTensor(),
])

dataset = ImageFolder(root=data_dir, transform=transform)


In [18]:
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [19]:
train_batch, label_batch = next(iter(dataloader))

In [20]:
new_dataset = []
for i, l in zip(train_batch, label_batch):
    new_dataset.append((i,l))
    
new_dataloader = DataLoader(new_dataset, batch_size=8, shuffle=True)

In [121]:
import torch
import torch.nn as nn

class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        
        # Define the feature extraction backbone (equivalent to VGG16)
        self.backbone = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Define the classification head
        self.classification_head = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(512, 1),  # Change the output dimension to 1
            nn.Sigmoid()
        )
        
        # Define the regression head
        self.regression_head = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(512, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 4),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        # Backbone feature extraction
        features = self.backbone(x)
        
        # Classification head
        class_output = self.classification_head(features)
        
        # Regression head
        regress_output = self.regression_head(features)
        
        return class_output, regress_output

# Create an instance of the model
model = CustomModel()
model

CustomModel(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, di

In [23]:
from helper_functions import accuracy_fn
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.01)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

CustomModel(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classification_head): Sequential(
    (0): AdaptiveAvgPool2d(output_size=(1, 1))
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=64, out_features=1, bias=True)
    (3): Sigmoid()
  )
  (regression_head): Sequential(
    (0): AdaptiveAvgPool2d(output_size=(1, 1))
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=512, out_features=4, bias=True)
    (5): Softmax(dim=1)
  )
)

In [32]:
img, label = next(iter(new_dataloader))


logit = model(img)
label = label.to(torch.float)
loss = loss_fn(logit.squeeze(),label)
pred = torch.round(torch.sigmoid(logit))
acc = accuracy_fn(label, pred.squeeze())
print("loss -- ",loss)
print("\n\n\n")
print("pred -- ", pred.squeeze())
print("\n\n\n")
print("true label -- ", label)
print("\n\n\n")
print("acc -- ", acc)

loss --  tensor(0.6616, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)




pred --  tensor([1., 1., 1., 1., 1., 1., 1., 1.], grad_fn=<SqueezeBackward0>)




true label --  tensor([1., 1., 0., 1., 1., 0., 1., 0.])




acc --  62.5


In [34]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    train_loss, train_acc = 0, 0
    model.to(device)
    for batch, (X, y) in enumerate(data_loader):
        # Send data to GPU
        X, y = X.to(device), y.to(device)
        y = y.to(torch.float)
        # 1. Forward pass
        y_pred_logits = model(X)
        y_pred = torch.round(torch.sigmoid(y_pred_logits)).squeeze().to(device)
        # 2. Calculate loss
        loss = loss_fn(y_pred_logits.squeeze(), y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.to(device)
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode(): 
        for X, y in data_loader:
            # Send data to GPU
            X, y = X.to(device), y.to(device)
            y = y.to(torch.float)
            # 1. Forward pass
            test_pred_logits = model(X)
            test_pred = torch.round(torch.sigmoid(test_pred_logits)).squeeze().to(device)
            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred_logits.squeeze(), y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred # Go from logits -> pred labels
            )
        
        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [41]:
torch.manual_seed(2)
from tqdm import tqdm
# Measure time

# Train and test model 
epochs = 3
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train_step(data_loader=new_dataloader, 
        model=model, 
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn,
        device=device
    )

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Epoch: 0
---------


 33%|████████████████████████████                                                        | 1/3 [00:33<01:06, 33.16s/it]

Train loss: 0.73727 | Train accuracy: 46.88%
Epoch: 1
---------


 67%|████████████████████████████████████████████████████████                            | 2/3 [00:57<00:28, 28.17s/it]

Train loss: 0.73621 | Train accuracy: 46.88%
Epoch: 2
---------


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:23<00:00, 27.75s/it]

Train loss: 0.73510 | Train accuracy: 46.88%


In [40]:

img.to(device)
label.to(device)

logit = model(img).to(device)
label = label.to(torch.float).to(device)
loss = loss_fn(logit.squeeze().to(device),label)
pred = torch.round(torch.sigmoid(logit))
acc = accuracy_fn(label, pred.squeeze())
print("loss -- ",loss)
print("\n\n\n")
print("pred -- ", pred.squeeze())
print("\n\n\n")
print("true label -- ", label)
print("\n\n\n")
print("acc -- ", acc)

loss --  tensor(0.6615, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)




pred --  tensor([1., 1., 1., 1., 1., 1., 1., 1.], grad_fn=<SqueezeBackward0>)




true label --  tensor([1., 1., 0., 1., 1., 0., 1., 0.])




acc --  62.5
